Import the following packages

In [1]:
import keras
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import mlflow
from mlflow.models import infer_signature

set the MLFLOW_TRACKING_URI environment variable to the URI of our MLflow tracking server:

In [2]:
# export MLFLOW_TRACKING_URI="http://localhost:5000"
mlflow.set_tracking_uri("http://127.0.0.1:8080")

load the dataset and split it into training, validation, and test sets.

In [3]:
# Load dataset
data = pd.read_csv(
    "https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",
    sep=";")

# Split the data into training, validation, and test sets
train, test = train_test_split(data, test_size=0.25, random_state=42)
train_x = train.drop(["quality"], axis=1).values
train_y = train[["quality"]].values.ravel()
test_x = test.drop(["quality"], axis=1).values
test_y = test[["quality"]].values.ravel()
train_x, valid_x, train_y, valid_y = train_test_split(
    train_x, train_y, test_size=0.2, random_state=42)
signature = infer_signature(train_x, train_y)

"train_model" function uses MLflow to track the parameters, results, and model itself of each trial as a child run. \n
"objective" function takes in the hyperparameters and returns the results of the train_model function for that set of hyperparameters.

In [8]:
# coefficient of determination (R^2) for regression
def r_square(y_true, y_pred):
    SS_res =  keras.backend.sum(keras.backend.square(y_true - y_pred)) 
    SS_tot = keras.backend.sum(keras.backend.square(y_true - keras.backend.mean(y_true))) 
    return (1 - SS_res/(SS_tot + keras.backend.epsilon()))

def train_model(params, data):
    # Specify model
    model = keras.Sequential()
    model.add(keras.Input([data['train_x'].shape[1]]))
    model.add(keras.layers.Normalization(mean=np.mean(data['train_x']), variance=np.var(data['train_x'])))

    # hidden layers
    for i in range(params['num_layers']):
        model.add(keras.layers.Dense(units=params['hidden_size'], activation='relu'))
        
    # Output layer
    model.add(keras.layers.Dense(1, activation='linear'))

    # Compile model
    model.compile(
        optimizer=keras.optimizers.SGD(learning_rate=params["lr"], momentum=params["momentum"]),
        loss="mean_squared_error", metrics=[keras.metrics.RootMeanSquaredError(), r_square])

    # Train model with MLflow tracking
    
    model.fit(
            train_x, train_y, validation_data=(data['valid_x'], data['valid_y']),
            epochs=params['epochs'], batch_size=params['batch_size'])
    # Evaluate the model
    eval_result = model.evaluate(data['valid_x'], data['valid_y'], batch_size=params['batch_size'])
    
    eval_rmse = eval_result[1]
    eval_r2 = eval_result[2]
    # Log parameters and results
    return {"loss": eval_rmse, "R2": eval_r2, "status": STATUS_OK, "model": model}


# do hyperparameter search

why this? MLflow has an internal capability to log every reading and thus it can record the hyperparameters for each run

In [14]:
mlflow.set_experiment("WineQuality_R03")
with mlflow.start_run():
    # Define the search space
    params = {
                "lr": np.random.choice(np.linspace(0.001, 0.1, 10)), 
                "momentum": np.random.choice(np.linspace(0.1, 0.99, 10)), 
                "epochs": np.random.choice([10, 100, 500, 1000]),
                "num_layers": np.random.choice([2, 3, 4, 5]),
                "batch_size": np.random.choice([32, 64, 128, 256]),
                "hidden_size": np.random.choice([1, 2, 3, 4, 5])
            }
    print(params)
    # data dict
    data = {
            'train_x': train_x, 'train_y': train_y,
            'valid_x': valid_x, 'valid_y': valid_y,
            'test_x': test_x, 'test_y': test_y
            }
    # store results
    result = train_model(params, data=data)

    signature = infer_signature(valid_x, result['model'].predict(valid_x))

    # Log the best parameters, loss, and model
    mlflow.log_params(params)
    mlflow.log_metric("eval_rmse", result["loss"])
    mlflow.log_metric("eval_r2", result["R2"])
    mlflow.tensorflow.log_model(model = result["model"], 
                                artifact_path="model", 
                                signature=signature)


{'lr': 0.034, 'momentum': 0.1, 'epochs': 100, 'num_layers': 4, 'batch_size': 64, 'hidden_size': 3}
Epoch 1/100
46/46 [==============================] - 1s 4ms/step - loss: 7.0135 - root_mean_squared_error: 2.6483 - r_square: -8.5727 - val_loss: 0.8295 - val_root_mean_squared_error: 0.9107 - val_r_square: -0.0865
Epoch 2/100
46/46 [==============================] - 0s 1ms/step - loss: 0.7939 - root_mean_squared_error: 0.8910 - r_square: -0.0214 - val_loss: 0.7831 - val_root_mean_squared_error: 0.8849 - val_r_square: -0.0183
Epoch 3/100
46/46 [==============================] - 0s 999us/step - loss: 0.7881 - root_mean_squared_error: 0.8878 - r_square: -0.0158 - val_loss: 0.7826 - val_root_mean_squared_error: 0.8846 - val_r_square: -0.0174
Epoch 4/100
46/46 [==============================] - 0s 994us/step - loss: 0.7884 - root_mean_squared_error: 0.8879 - r_square: -0.0186 - val_loss: 0.7831 - val_root_mean_squared_error: 0.8850 - val_r_square: -0.0185
Epoch 5/100
46/46 [==================

INFO:tensorflow:Assets written to: /var/folders/83/j83q5_mj11956_7k9tnld0ym0000gn/T/tmpmou9z1nv/model/data/model/assets
2024/04/23 11:18:31 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: /var/folders/83/j83q5_mj11956_7k9tnld0ym0000gn/T/tmpmou9z1nv/model, flavor: tensorflow). Fall back to return ['tensorflow==2.13.0', 'cloudpickle==3.0.0']. Set logging level to DEBUG to see the full traceback. 
